In [ ]:
!ollama show embeddinggemma:latest

In [1]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain_ollama.embeddings import OllamaEmbeddings

In [2]:
load_dotenv()

os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
llm_model = ChatGroq(model="openai/gpt-oss-20b", temperature=0.5)
embedding_model = OllamaEmbeddings(model="embeddinggemma:latest")

In [6]:
def get_embedding(text, embedding_model, input_type='text'):
    response = embedding_model.embed_query(text)
    return response
    

In [7]:
get_embedding("Hola Amigos", embedding_model=embedding_model)

[-0.17553535,
 -0.050903402,
 -0.019111441,
 -0.02212273,
 0.012935456,
 0.0130672045,
 -0.002526733,
 0.027326709,
 -0.031671993,
 -0.07433304,
 -0.006454614,
 -0.048487816,
 0.04779181,
 -0.078359745,
 0.06395146,
 0.00570887,
 -0.03132466,
 -0.03412367,
 -0.072620474,
 -0.02913309,
 -0.0029708652,
 0.0028103874,
 0.009727291,
 -0.033652887,
 0.058007773,
 0.023762815,
 0.032191228,
 0.050254393,
 -0.020006007,
 0.007039961,
 -0.036812246,
 0.00574755,
 0.047886677,
 0.017444322,
 0.019296076,
 0.017860241,
 0.0013491893,
 -0.03531216,
 0.08846473,
 -0.04525587,
 -0.04441781,
 -0.007822892,
 -0.049112514,
 -0.031205356,
 -0.033728953,
 -0.0006381876,
 -0.0407417,
 -0.051715035,
 0.03372487,
 0.020762092,
 0.0012762122,
 -0.01138861,
 -0.082426555,
 0.017695373,
 0.009239983,
 -0.022052987,
 -0.0076148696,
 0.019701201,
 -0.06068222,
 0.024361815,
 -0.009422922,
 -0.010502905,
 -0.010187802,
 -0.031770233,
 0.035948038,
 0.009783509,
 -0.037941895,
 -0.040331785,
 0.028253483,
 0.1693